In [5]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

#from google.colab import drive
#drive.mount('/content/gdrive')

In [7]:
#Q1. Execute the following statement. What is displayed? What does it mean?
#'!pwd' shows the directory where the user is working in. 
# When we run this code in Colab without mounting to the drive, we will find that '/content' is displayed.
!pwd

/content


In [4]:
#Q2: Execute the following statement. What happens?  Examine the left column of your colab page to see what happens.
''' As we execute the statement, the new folder, course-v3, is included in the left column of my colab page.'''
!git clone https://github.com/fastai/course-v3.git

Cloning into 'course-v3'...
remote: Enumerating objects: 5890, done.
remote: Total 5890 (delta 0), reused 0 (delta 0), pack-reused 5890
Receiving objects: 100% (5890/5890), 263.03 MiB | 40.83 MiB/s, done.
Resolving deltas: 100% (3249/3249), done.


In [8]:
#Q3. Execute the following statement. What are displayed by the execution of sys.path? What do it mean?
'''Executing 'sys.path', we can find a list of several directories including the current working directory. 
When we import a module, python internally explore the directories in 'sys.path' to find the module.
By appending "/content/course-v3/nbs/dl2" to sys.path, we can import modules from packages in the directory. '''
import sys
sys.path.append('/content/course-v3/nbs/dl2')
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/course-v3/nbs/dl2']

In [9]:
#export
#Q4: "from exp.nb_01 import *" imports classes, functions, etc from module nb_01 in package exp.
# Q4a.  How do you know that exp is a package?
'''If a file "__init__.py" is in a folder, the folder is regarded as a package. We know that exp is a package because 
it has an "__init__.py" file  '''

# Q4b.   How is the Python system able to get access to package exp? That is, how can it search for it?
# The python system can get access to the package exp bacause we appended the absolute path of the package to sys.path in Q3. 

from exp.nb_02 import *
import torch.nn.functional as F

## Initial setup

### Data

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=1786)

In [10]:
from IPython.display import Image
from six.moves import urllib

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

def get_data():
    import os
    import torchvision.datasets as datasets
    root = '../data'
    if not os.path.exists(root):
        os.mkdir(root)

    train_set = datasets.MNIST(root=root, train=True, download=True)
    test_set = datasets.MNIST(root=root, train=False, download=True) #load validation set
    x_train, x_valid = train_set.data.split([50000, 10000])
    y_train, y_valid = train_set.targets.split([50000, 10000])
    return (x_train.view(50000, -1) / 256.0), y_train.float(), (x_valid.view(10000, -1))/ 256.0, y_valid.float()



In [11]:
mpl.rcParams['image.cmap'] = 'gray'

In [12]:
#Q5: when you execute the following statement, where is the downloaded data stored? Examine the left column of your colab page.
'''
The downloaded data is stored in a "/data" folder. As we know from '!pwd', we know that current working directory is "/content".
In get_data functions, we made a folder "/data" in "/" using ".." and mkdir, and specified "/data" as a place to have MNIST datasets.
'''

x_train,y_train,x_valid,y_valid = get_data()


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [13]:
y_train.shape

torch.Size([50000])

In [14]:
#Q6a: Execute the following statement. What does the number displayed mean?
# The number means the number of rows in x_train data. It also reveals the number of samples in x_train data.
len(x_train)

50000

In [15]:
#Q6b: Execute the following statement. What does the numbers displayed mean?
'''The number "5" means a label for the first sample. For MNIST dataset, we have numbers from 0 to 9 as a label of the dataset because
MNIST classification task is aimed at classifying the images of hand-written numbers into 10 numbers. 
y_train[0]

SyntaxError: ignored

In [ ]:
#Q7: Display the values of n,m, c, and nh. What are they? For what are they used in the following code? 
n,m = x_train.shape
c = y_train.max()+1
nh = 50

"""'n' and 'm' means the number of samples(50000) and features(784) of x_train data respectively. 
'c'(10) means the number of options in y_train because we have 0 to 9 in y_train. 
'nh' means the number of hidden nodes in the first layer that will be used later."""
n,m,c

In [ ]:
#Q8: The following defines Model class, which will be used to create a neural net.
#Q8a: What is nn.Module?, What is nn.Linear? What is nn.ReLU? 
"""nn.Module is a base class that has many useful attributes and functions for making new layers or models. Normally, 
we inherit those attributes and functions from nn.Module when we define new classes used to construct neural network. 
nn.Linear is a class that applies linear transformation to the input matrix 
and nn.ReLU is a class that applies ReLU functions to the components of input matrix. 
"""
#Q8b. What is the difference between  nn.Linear itself and nn.Linear(n_in,nh) ? 
'''Whereas nn.Linear is a class, nn.Linear(n_in,nh) is an instance of nn.Linear that has information about 
the number of input features and hidden nodes which is used to make weight matrixes.'''

class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
nn.Linear

In [ ]:
nn.Linear(50,20)

In [ ]:
#Q9a. When Model(m,nh, 10) is executed, which method in Model class is executed?
model = Model(m, nh, 10)
''' When the codes are executed, the "__init__" function is executed. 
The function makes the instance variables that will be used when '__call__' method is executed.'''

In [ ]:
#Q10a. Is model a function or an object? If it is an object, how is it used as if it is a function?
# When model(x_train) is executed, which method in Model is executed. What does this method create? 
pred = model(x_train)
'''When the codes are executed, "__call__" method will be executed.
 "__call__" takes x_train data, applies several layers defined by "__init__" function, and returns the output(predictions) of the neural net.'''

In [ ]:
#Q11: Execute the following statement. What does the displayed numbers mean?
pred[0]
'''The numbers are the predictions of the first sample. It has 10 numbers because we specified 10 as the number of output. 
10 numbers means how probable the sample belong to each class.(But, It does not mean "probability") 
The bigger the number is, the more likely the sample belongs to that class. 
For example, if we have the biggest number in fifth index for the first samle,
the prediction says that first sample would belong to the fifth class, 4. '''

In [ ]:
#Q12. Execute the following statement. Is the resulting value near to 1? If not, what does it imply?
pred[0].sum()
'''No, the resulting value is not close to 1. It implies that
 we have to make some changes to the numbers in order to use them like a probability. '''

### Cross entropy loss

First, we will need to compute the softmax of our activations. This is defined by:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

or more concisely:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum_{0 \leq j \leq n-1} e^{x_{j}}}$$ 

In practice, we will need the log of the softmax when we calculate the loss.

In [ ]:
def softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True)))

In [ ]:
sm_pred = softmax(pred)


In [ ]:
#Q13. Execute the following statement. What does the displayed numbers mean?
sm_pred[0]
''' The numbers are the result of softmax function applied to predictions of the first sample. 
Unlike the case of Q11, the numbers ranges from 0 to 1 and mean how probable the sample belong to each class.
The bigger the number is, the more likely the sample belongs to that class.'''

In [ ]:
#Q14. Execute the following statement. What does the displayed numbers mean?
sm_pred[0].sum()
'''It is 1, which means that numbers in sm_pred has probability-like numbers 
that gives information about how probable each sample belongs to 10 classes.'''

In [ ]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

In [ ]:
log_sm_pred = log_softmax(pred)

In [ ]:
log_sm_pred[0]

In [ ]:
log_sm_pred[0].sum()



## Q15. Read the following paragraph to understand what the cross entroy loss functioni is. 

The difference between probabilities interpretation: https://datascience.stackexchange.com/questions/20296/cross-entropy-loss-explanation

The cross entropy formula takes in two distributions,  $p(x^{(s)})$, the true distribution (defined by the label data), and $\hat{p}(x^{(s)})$, the estimated distribution (predicted by the neural net), defined over the discrete variable x and is given by

$H(p,\hat{p})=−\sum_{s \in B} p(x^{(s)}) \cdot log(\hat{p}(x^{(s)}))$


In general, $ p(x^{(s)}) = [ p_{1} (x^{(s)}), ..., p_{n}(x^{(s)})]$ is a probability distribution over a set of categories. 
But since our $ p(x^{(s)})$ are 1-hot encoded, that is, in the form of  $ p(x^{(s)}) =[0,0,..,0,1,0..]$, where the probability of only one category is one and those of the other categories are all zero, this can be rewritten as

 $-\sum_{s \in B} [ 0*\log(\hat{p}_{1} (x^{(s)}))
 + 1*\log(\hat{p}_{i} (x^{(s)}) ) +..+0*\log(\hat{p}_{n} (x^{(s)}) ) ] \\
  = -\sum_{s \in B} \log(\hat{p}_{i(s)} ) (x^{(s)}) ) 
  \tag{crossEntroyEq}$ 

#  In this notebook, the softmax function plays the role of the probability distribution $\hat{p}_{1} (x^{(s)})$.

  Here $i(s)$ is the index of the one-hot  probability distribution $p(x^{(s)})$ where the probability is one. 
 
 

This can be done using numpy-style [integer array indexing](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html#integer-array-indexing). Note that PyTorch supports all the tricks in the advanced indexing methods discussed in that link.

In [ ]:
#Q15a. Execute the following statement. What do the displayed numbers mean? 
y_train[:3]
'''The numbers mean the true labels of the first three samples in an integer format, not in an 1-hot encoded format. 
We can change the numbers in y_train into 1-hot encoded format. For example, we can change "5" into [0,0,0,0,0,1,0,0,0,0]. 
The 1-hot encoded number can represent probability distribution of true labels over several categories, where the probability of one category is one and those of the other categories are zero.'''

In [17]:
#Q15b.  Execute the following statement. What do the displayed numbers mean? 
log_sm_pred[ [0,1,2]]
'''The three 1 dimension tensors for 3 samples in "log_sm_pred[[0,1,2]]" have 10 numbers which means the log_softmax values corresponding to each classes. 
 The softmax values without log can represent the estimated probability distribution predicted by our model.
In this case, log_sm_pred is logged softmax values which can be used to calculate cross entropy. '''

NameError: ignored

In [ ]:
#Q15c.  Execute the following statement. What do the displayed numbers mean? Explain compare the result of Q15b and the result of this statement.
log_sm_pred[[0,1,2], [5,0,4]]
'''Each number means the log_softmax values corresponding to the true values. 
When we calculate cross entropy, we multiply estimated logged probability of belonging to each class with actual probability of beloning to each class. 
We can get the actual probability from the y_train in 1-hot encoded format. It becomes 1 only for the answer class and 0 for the other classes.
Even though we should multiply numbers all classes, we can rule out the case when we multiply 0 with logged probability. 
If we elimate whole the case, only the log_softmax values corresponding to the true values are left 
From the results of Q15b, we can calculate cross entropy without minus just by taking the log_softmax values corresponding to the true labels using integer array indexing.'''

In [ ]:
y_train.shape[0]

In [ ]:
y_train

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=2081)

In [ ]:
log_sm_pred[0]

In [ ]:
range(y_train.long()[0:10].shape[0])

In [ ]:
log_sm_pred[ range(y_train.long()[0:10].shape[0]), y_train.long()[0:10] ]

In [ ]:
def nll(input, target):
   
   return -input[range(target.shape[0]), target].mean()

In [ ]:
y_train.long().shape[0]

In [ ]:
x_train.shape

In [ ]:
#Q16. Read nill function and explain how this function computes the result. You need to refer to {CrossEntropyLoss}.
loss = nll(log_sm_pred, y_train.long())
'''In nll, we calculated cross entropy of each sample and attached minus to the result, which became CrossEntropy.
By calculating the mean of CrossEntropy from all samples, we can get {CrossEntropyLoss}.'''

In [ ]:
y_train

In [ ]:
y_train.long()

In [ ]:
loss

Note that the formula 

$$\log \left ( \frac{a}{b} \right ) = \log(a) - \log(b)$$ 

gives a simplification when we compute the log softmax, which was previously defined as `(x.exp()/(x.exp().sum(-1,keepdim=True))).log()`

In [ ]:
def log_softmax(x): return x - x.exp().sum(-1,keepdim=True).log()

In [ ]:
test_near(nll(log_softmax(pred), y_train.long()), loss)

Then, there is a way to compute the log of the sum of exponentials in a more stable way, called the [LogSumExp trick](https://en.wikipedia.org/wiki/LogSumExp). The idea is to use the following formula:

$$\log \left ( \sum_{j=1}^{n} e^{x_{j}} \right ) = \log \left ( e^{a} \sum_{j=1}^{n} e^{x_{j}-a} \right ) = a + \log \left ( \sum_{j=1}^{n} e^{x_{j}-a} \right )$$

where a is the maximum of the $x_{j}$.

In [ ]:
def logsumexp(x):
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()

This way, we will avoid an overflow when taking the exponential of a big activation. In PyTorch, this is already implemented for us. 

In [ ]:
test_near(logsumexp(pred), pred.logsumexp(-1))

So we can use it for our `log_softmax` function.

In [ ]:
def log_softmax(x): return x - x.logsumexp(-1,keepdim=True)

In [ ]:
test_near(nll(log_softmax(pred), y_train.long()), loss)

Then use PyTorch's implementation.

In [ ]:
test_near(F.nll_loss(F.log_softmax(pred, -1), y_train.long()), loss)

In PyTorch, `F.log_softmax` and `F.nll_loss` are combined in one optimized function, `F.cross_entropy`.

In [ ]:
test_near(F.cross_entropy(pred, y_train.long()), loss)

## Basic training loop

Basically the training loop repeats over the following steps:
- get the output of the model on a batch of inputs
- compare the output to the labels we have and compute a loss
- calculate the gradients of the loss with respect to every parameter of the model
- update said parameters with those gradients to make them a little bit better

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=2542)

In [ ]:
loss_func = F.cross_entropy # This criterion combines log_softmax and nll_loss in a single function.

In [ ]:
#export
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

In [ ]:
bs=64                  # batch size

xb = x_train[0:bs]     # a mini-batch from x
preds = model(xb)      # predictions
preds[0], preds.shape

In [ ]:
yb = y_train[0:bs]
loss_func(preds, yb.long())

In [ ]:
accuracy(preds, yb)

In [ ]:
lr = 0.5   # learning rate
epochs = 1 # how many epochs to train for

# The mechanism for training the network: (1) computing the graidents of Loss with respect to tensors at each layer, (2) backpropgation: applying the chain rule to compute the gradient vector of the parameters, (3) updating the parameters.

https://github.com/pytorch/pytorch/blob/35bd2b3c8b64d594d85fc740e94c30aa67892a34/torch/tensor.py

https://github.com/pytorch/pytorch/blob/35bd2b3c8b64d594d85fc740e94c30aa67892a34/torch/tensor.py

https://stackoverflow.com/questions/57248777/backward-function-in-pytorch
: Pytorch does not support this non-scalar function derivatives. Instead, pytorch assumes out is only an intermediate tensor and somewhere "upstream" there is a scalar loss function, that through chain rule provides d loss/ d out[i,j]. This "upstream" gradient is of size 2-by-3 and this is actually the argument you provide backward in this case: out.backward(g) where g_ij = d loss/ d out_ij.

The gradients are then calculated by chain rule d loss / d a[i,j] = (d loss/d out[i,j]) * (d out[i,j] / d a[i,j]); In fact,  So it's not just (d loss/d out[i,j]) * (d out[i,j] / d a[i,j]), but in fact sum_{k,l} (d loss/d out[k,l]) * (d out[k,l] / d a[i,j]).

# The "gradient accumulation"

https://discuss.pytorch.org/t/why-do-we-need-to-set-the-gradients-manually-to-zero-in-pytorch/4903/9

In the following example, y.backward() is called 5 times, so the final value of x.grad will be 5*cos(0)=5.

import torch
from torch.autograd import Variable

x = Variable(torch.Tensor([[0]]), requires_grad=True)

for t in range(5):
    y = x.sin() 
    y.backward()
    
print(x.grad) # shows 5
Calling x.grad.data.zero_() before y.backward() can make sure x.grad is exactly the same as current y’(x), not a sum of y’(x) in all previous iterations.

x = Variable(torch.Tensor([[0]]), requires_grad=True) 

for t in range(5):
    if x.grad is not None:
        x.grad.data.zero_()
    y = x.sin() 
    y.backward()

print(x.grad) # shows 1
I also got confused by this “zeroing gradient” when first learning pytorch. The doc of torch.autograd.backward does mention that

This function accumulates gradients in the leaves - you might need to zero them before calling it.

But this is quite hard to find and pretty confusing for (say) tensorflow users.

Official tutorials like 60 Minute Blitz 45 or PyTorch with Examples 69 both say nothing about why one needs to call grad.data.zero_() during training. I think it would be useful to explain this a little more in beginner-level tutorials. RNN is a good example for why accumulating gradient (instead of refreshing) is useful, but I guess new users wouldn’t even know that backward() is accumulating gradient


# A more rigorous reference to autograd:

https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/

torch.no_grad():

When we are computing gradients, we need to cache input values, and intermediate features as they maybe required to compute the gradient later.

The gradient of  
b
=
w
1
∗
a
 w.r.t it's inputs 
w
1
 and 
a
 is 
a
 and 
w
1
 respectively. We need to store these values for gradient computation during the backward pass. This affects the memory footprint of the network.

While, we are performing inference, we don't compute gradients, and thus, don't need to store these values. Infact, no graph needs to be create during inference as it will lead to useless consumption of memory.

PyTorch offers a context manager, called torch.no_grad for this purpose.


with torch.no_grad:

	inference code goes here 

No graph is defined for operations executed under this context manager.




In [ ]:
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()
bs=64                  # batch size

xb = x_train[0:bs]     # a mini-batch from x
cost_function = F.cross_entropy

In [ ]:
lr = 0.5   # learning rate
epochs = 2 # how many epochs to train for

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1): # for each batch in the current epoch
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]

        ybHat = model(xb)
        h = ybHat.register_hook( lambda grad: print (grad) )

        loss = loss_func( ybHat, yb.long())

        # loss.backward() computes the gradient of loss w.r.t. graph LEAVES.
        #  This function accumulates gradients in the leaves - you might need to zero
        # ``.grad`` attributes or set them to ``None`` before calling it.
        #  (This makes it easier for rnn, because each module will be back propogated through several times.)
       
       # Attribute grad of tensor ybHat (ybHat.grad = dloss/dybHat) is ``None`` by default and becomes a Tensor the first time a call to
        #func `backward` computes gradients for ybHat.
        #The attribute will then contain the gradients computed and future calls to
        #:func:`backward` will accumulate (add) gradients into it.

        #Since the backward() function accumulates gradients, and you don’t want to mix up gradients between minibatches, 
        # you have to zero them out at the start of a new minibatch.
       


        ybHat.grad
        loss.backward() # When you call loss.backward(), all it does is compute gradient of loss 
                        # w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter

        ybHat.grad                
        #optimizer.step() updates all the parameters based on parameter.grad 
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias   -= l.bias.grad   * lr

                    #we don't care about gradients from the previous batch. Not zeroing grads would lead to gradient accumulation across batches
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()

In [ ]:
loss_func(model(xb), yb.long()), accuracy(model(xb), yb)

## Using parameters and optim

### Parameters

Use `nn.Module.__setattr__` and move relu to functional:

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=2818)

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
        
    def __call__(self, x): return self.l2(F.relu(self.l1(x)))

In [ ]:
model = Model(m, nh, 10)

In [ ]:
for name,l in model.named_children(): print(f"{name}: {l}")

In [ ]:
model

In [ ]:
model.l1

In [ ]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb.long())

            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
                model.zero_grad()

In [ ]:
fit()
loss_func(model(xb), yb.long()), accuracy(model(xb), yb)

Behind the scenes, PyTorch overrides the `__setattr__` function in `nn.Module` so that the submodules you define are properly registered as parameters of the model.

In [ ]:
class DummyModule():
    def __init__(self, n_in, nh, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
        
    def __setattr__(self,k,v):
        if not k.startswith("_"): self._modules[k] = v
        super().__setattr__(k,v)
        
    def __repr__(self): return f'{self._modules}'
    
    def parameters(self):
        for l in self._modules.values():
            for p in l.parameters(): yield p

In [ ]:
mdl = DummyModule(m,nh,10)
mdl

In [ ]:
[o.shape for o in mdl.parameters()]

### Registering modules

We can use the original `layers` approach, but we have to register the modules.

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=2997)

In [ ]:
layers = [nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10)]

In [ ]:
class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers
        for i,l in enumerate(self.layers): self.add_module(f'layer_{i}', l)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
model = Model(layers)

In [ ]:
model

### nn.ModuleList

`nn.ModuleList` does this for us.

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3173)

In [ ]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
model = SequentialModel(layers)

In [ ]:
model

In [ ]:
fit()
loss_func(model(xb), yb.long()), accuracy(model(xb), yb)

### nn.Sequential

`nn.Sequential` is a convenient class which does the same as the above:

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3199)

In [ ]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
fit()
loss_func(model(xb), yb.long()), accuracy(model(xb), yb)

In [ ]:
nn.Sequential??

In [ ]:
model

### optim

Let's replace our previous manually coded optimization step:

```python
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```

and instead use just:

```python
opt.step()
opt.zero_grad()
```

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3278)

In [ ]:
class Optimizer():
    def __init__(self, params, lr=0.5): self.params,self.lr=list(params),lr
        
    def step(self):
        with torch.no_grad():
            for p in self.params: p -= p.grad * lr

    def zero_grad(self):
        for p in self.params: p.grad.data.zero_()

In [ ]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
opt = Optimizer(model.parameters())

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb.long())

        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
loss,acc = loss_func(model(xb), yb.long()), accuracy(model(xb), yb)
loss,acc

PyTorch already provides this exact functionality in `optim.SGD` (it also handles stuff like momentum, which we'll look at later - except we'll be doing it in a more flexible way!)

In [ ]:
#export
from torch import optim

In [ ]:
optim.SGD.step??

In [ ]:
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))
    return model, optim.SGD(model.parameters(), lr=lr)

In [ ]:
model,opt = get_model()
loss_func(model(xb), yb.long())

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb.long())

        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
loss,acc = loss_func(model(xb), yb.long()), accuracy(model(xb), yb)
loss,acc

Randomized tests can be very useful.

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3442)

In [ ]:
assert acc>0.7

## Dataset and DataLoader

### Dataset

It's clunky to iterate through minibatches of x and y values separately:

```python
    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]
```

Instead, let's do these two steps together, by introducing a `Dataset` class:

```python
    xb,yb = train_ds[i*bs : i*bs+bs]
```

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3578)

In [ ]:
#export
class Dataset():
    def __init__(self, x, y): self.x,self.y = x,y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i],self.y[i]

In [ ]:
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
assert len(train_ds)==len(x_train)
assert len(valid_ds)==len(x_valid)

In [ ]:
xb,yb = train_ds[0:5]
assert xb.shape==(5,28*28)
assert yb.shape==(5,)
xb,yb

In [ ]:
model,opt = get_model()

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        xb,yb = train_ds[i*bs : i*bs+bs]
        pred = model(xb)
        loss = loss_func(pred, yb.long())

        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
loss,acc = loss_func(model(xb), yb.long()), accuracy(model(xb), yb)
assert acc>0.7
loss,acc

### DataLoader

Previously, our loop iterated over batches (xb, yb) like this:

```python
for i in range((n-1)//bs + 1):
    xb,yb = train_ds[i*bs : i*bs+bs]
    ...
```

Let's make our loop much cleaner, using a data loader:

```python
for xb,yb in train_dl:
    ...
```

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3674)

In [ ]:
class DataLoader():
    def __init__(self, ds, bs): self.ds,self.bs = ds,bs
    def __iter__(self):
        for i in range(0, len(self.ds), self.bs): yield self.ds[i:i+self.bs]

In [ ]:
train_dl = DataLoader(train_ds, bs)
valid_dl = DataLoader(valid_ds, bs)

In [ ]:
xb,yb = next(iter(valid_dl))
assert xb.shape==(bs,28*28)
assert yb.shape==(bs,)

In [ ]:
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
model,opt = get_model()

In [ ]:
def fit():
    for epoch in range(epochs):
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_func(pred, yb.long())
            loss.backward()
            opt.step()
            opt.zero_grad()

In [ ]:
fit()

In [ ]:
loss,acc = loss_func(model(xb), yb.long()), accuracy(model(xb), yb)
assert acc>0.7
loss,acc

### Random sampling

We want our training set to be in a random order, and that order should differ each iteration. But the validation set shouldn't be randomized.

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=3942)

In [ ]:
class Sampler():
    def __init__(self, ds, bs, shuffle=False):
        self.n,self.bs,self.shuffle = len(ds),bs,shuffle
        
    def __iter__(self):
        self.idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        for i in range(0, self.n, self.bs): yield self.idxs[i:i+self.bs]

In [ ]:
small_ds = Dataset(*train_ds[:10])

In [ ]:
s = Sampler(small_ds,3,False)
[o for o in s]

In [ ]:
s = Sampler(small_ds,3,True)
[o for o in s]

In [ ]:
def collate(b):
    xs,ys = zip(*b)
    return torch.stack(xs),torch.stack(ys)

class DataLoader():
    def __init__(self, ds, sampler, collate_fn=collate):
        self.ds,self.sampler,self.collate_fn = ds,sampler,collate_fn
        
    def __iter__(self):
        for s in self.sampler: yield self.collate_fn([self.ds[i] for i in s])

In [ ]:
train_samp = Sampler(train_ds, bs, shuffle=True)
valid_samp = Sampler(valid_ds, bs, shuffle=False)

In [ ]:
train_dl = DataLoader(train_ds, sampler=train_samp, collate_fn=collate)
valid_dl = DataLoader(valid_ds, sampler=valid_samp, collate_fn=collate)

In [ ]:
xb,yb = next(iter(valid_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
xb,yb = next(iter(train_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
xb,yb = next(iter(train_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
model,opt = get_model()
fit()

loss,acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc>0.7
loss,acc

### PyTorch DataLoader

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=4171)

In [ ]:
#export
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [ ]:
train_dl = DataLoader(train_ds, bs, sampler=RandomSampler(train_ds), collate_fn=collate)
valid_dl = DataLoader(valid_ds, bs, sampler=SequentialSampler(valid_ds), collate_fn=collate)

In [ ]:
model,opt = get_model()
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

PyTorch's defaults work fine for most things however:

In [ ]:
train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [ ]:
model,opt = get_model()
fit()

loss,acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc>0.7
loss,acc

Note that PyTorch's `DataLoader`, if you pass `num_workers`, will use multiple threads to call your `Dataset`.

## Validation

You **always** should also have a [validation set](http://www.fast.ai/2017/11/13/validation-sets/), in order to identify if you are overfitting.

We will calculate and print the validation loss at the end of each epoch.

(Note that we always call `model.train()` before training, and `model.eval()` before inference, because these are used by layers such as `nn.BatchNorm2d` and `nn.Dropout` to ensure appropriate behaviour for these different phases.)

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=4260)

In [ ]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb.long())
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb.long())
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

*Question*: Are these validation results correct if batch size varies?

`get_dls` returns dataloaders for the training and validation sets:

In [ ]:
#export
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

Now, our whole process of obtaining the data loaders and fitting the model can be run in 3 lines of code:

In [ ]:
train_dl,valid_dl = get_dls(train_ds, valid_ds, bs)
model,opt = get_model()
loss,acc = fit(5, model, loss_func, opt, train_dl, valid_dl)

In [ ]:
assert acc>0.9

## Export

In [ ]:
!python notebook2script.py 03_minibatch_training.ipynb